In [1]:
import sys
sys.path.append('../')

from dataset import*
from synthetic_concept_model import *
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Data generation

In [2]:
feature_dim_info = dict()
label_dim_info = dict()
transform_dim = 100000

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6
num_concepts = 2
transforms_2concept = None
transforms_2hd = None
num_data = 1000
total_data, total_labels, total_concepts, total_raw_features = generate_data_concepts(num_data, num_concepts,
                                                                                      feature_dim_info,
                                                                                      label_dim_info,
                                                                                      transform_dim=transform_dim,
                                                                                     noise=0.5,
                                                                                     pos_prob=0.7)

In [3]:
# Data splitting & loading
dataset = MultiConcept(total_data, total_labels, total_concepts, 0)
batch_size = 100
trainval_dataset, test_dataset = torch.utils.data.random_split(dataset,  
                                                            [int(0.5 * num_data), num_data - int(0.5 * num_data)])
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset,
                                                           [int(0.8 * len(trainval_dataset)), len(trainval_dataset) - int(0.8 * len(trainval_dataset))])

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                          batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False)

### Experiment 1
One known concept $c_1$ derived from information components $W_{U_1}, W_s$. Label $Y$ is composed of information components $y=f(W_{U_1}, W_s, W_{U_2})$. We try to recover $W_{U_2}$ by $\arg \max_{Z_x} I(Z_x;Y|Z_{c_1})$, assuming that $Z_{c_1}$ represents $\{W_{U_1}, W_s\}$


In [5]:
# models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_dim = 512
embed_dim = 50
concept_encoder = ConceptEncoder(transform_dim, embed_dim, 1, hidden_dim).to(device)
model = ConceptCLSUP(transform_dim, embed_dim, 2, hidden_dim, embed_dim).to(device)

In [6]:
# train concpet encoder
trained_concept_encoder = train_concept_encoder(concept_encoder, train_loader,val_loader, transform_dim, device, 1e-3, 1e-5, 25, 3, '../trained_models')

Train Epoch: 0 [0/400 (0%)]	Concept Loss: 2.218029
Val loss: 9.472379
Train Epoch: 1 [0/400 (0%)]	Concept Loss: 9.272290
Train Epoch: 2 [0/400 (0%)]	Concept Loss: 10.312690
Train Epoch: 3 [0/400 (0%)]	Concept Loss: 2.059592
Val loss: 1.172986
Train Epoch: 4 [0/400 (0%)]	Concept Loss: 1.456407
Train Epoch: 5 [0/400 (0%)]	Concept Loss: 1.675846
Train Epoch: 6 [0/400 (0%)]	Concept Loss: 1.062924
Val loss: 3.072671
Val loss did not improve


In [7]:
# train concept informed model
trained_concept_informed_model = train_concept_informed_model(trained_concept_encoder, model, train_loader, val_loader, 100, device, 1e-4, 25, 3, '../trained_models')

Train Epoch: 0 [0/400 (0%)]	Concept Loss: -0.000050
Val loss: -0.084921
Train Epoch: 1 [0/400 (0%)]	Concept Loss: -0.093654
Train Epoch: 2 [0/400 (0%)]	Concept Loss: -0.426325
Train Epoch: 3 [0/400 (0%)]	Concept Loss: -0.907629
Val loss: -1.366242
Train Epoch: 4 [0/400 (0%)]	Concept Loss: -1.329677
Train Epoch: 5 [0/400 (0%)]	Concept Loss: -1.862257
Train Epoch: 6 [0/400 (0%)]	Concept Loss: -2.323284
Val loss: -2.507186
Train Epoch: 7 [0/400 (0%)]	Concept Loss: -2.514968
Train Epoch: 8 [0/400 (0%)]	Concept Loss: -2.642528
Train Epoch: 9 [0/400 (0%)]	Concept Loss: -2.948462
Val loss: -3.026505
Train Epoch: 10 [0/400 (0%)]	Concept Loss: -3.085452
Train Epoch: 11 [0/400 (0%)]	Concept Loss: -3.089523
Train Epoch: 12 [0/400 (0%)]	Concept Loss: -3.193561
Val loss: -3.247119
Train Epoch: 13 [0/400 (0%)]	Concept Loss: -3.312241
Train Epoch: 14 [0/400 (0%)]	Concept Loss: -3.325592
Train Epoch: 15 [0/400 (0%)]	Concept Loss: -3.571067
Val loss: -3.460700
Train Epoch: 16 [0/400 (0%)]	Concept Loss:

In [8]:
# Evaluation using linear logistic regression model

# Embeddings
train_embeds = trained_concept_encoder.get_embedding(torch.stack([sample[0] for sample in  train_dataset]).to(device)).detach().cpu().numpy()
train_labels = np.array([sample[-1].item() for sample in  train_dataset])

test_embeds = trained_concept_encoder.get_embedding(torch.stack([sample[0] for sample in  test_dataset]).to(device)).detach().cpu().numpy()
test_labels = np.array([sample[-1].item() for sample in  test_dataset])

# Train Logistic Classifier
clf = LogisticRegression(max_iter=1000).fit(train_embeds, train_labels)
predictions = clf.predict(test_embeds)

# Calculate and print accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

# Calculate and print precision
precision = precision_score(test_labels, predictions)
print("Precision:", precision)

# Calculate and print recall
recall = recall_score(test_labels, predictions)
print("Recall:", recall)

# Calculate and print F1-score
f1 = f1_score(test_labels, predictions)
print("F1-score:", f1)

Accuracy: 0.992
Precision: 0.974025974025974
Recall: 1.0
F1-score: 0.9868421052631579


### Baseline 1 (logistic regression on $x$)

In [9]:
# Embeddings
train_embeds = torch.stack([sample[0] for sample in  train_dataset]).detach().cpu().numpy()
train_labels = np.array([sample[-1].item() for sample in  train_dataset])

test_embeds = torch.stack([sample[0] for sample in  test_dataset]).detach().cpu().numpy()
test_labels = np.array([sample[-1].item() for sample in  test_dataset])

# Train Logistic Classifier
clf = LogisticRegression(max_iter=1000).fit(train_embeds, train_labels)
predictions = clf.predict(test_embeds)

# Calculate and print accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

# Calculate and print precision
precision = precision_score(test_labels, predictions)
print("Precision:", precision)

# Calculate and print recall
recall = recall_score(test_labels, predictions)
print("Recall:", recall)

# Calculate and print F1-score
f1 = f1_score(test_labels, predictions)
print("F1-score:", f1)

Accuracy: 0.988
Precision: 0.9736842105263158
Recall: 0.9866666666666667
F1-score: 0.9801324503311258


In [10]:
sum(train_labels)/len(train_labels)

0.27